# Machine Learnig Bootcamp
# HomeWork 3

Getting the data,
For this Homework,

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Data = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [2]:
Data.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [7]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


## Data preparation
Check if the missing values are presented in the features.
If there are missing values:
For caterogiral features, replace them with 'NA'
For numerical features, replace with with 0.0
Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution. Use train_test_split function for that with random_state=1

In [9]:
missing_values = Data.isnull().sum().loc[lambda x : x > 0].shape[0]
print(f'The number of features with missing values is {missing_values}')


The number of features with missing values is 5


In [10]:
if missing_values > 0:
    categorical = Data.select_dtypes(include=['object']).columns
    numerical = Data.select_dtypes(include=['number']).columns

    Data[categorical] = Data[categorical].fillna('NA')
    Data[numerical] = Data[numerical].fillna(0.0)

In [11]:
missing_values = Data.isnull().sum().loc[lambda x : x > 0].shape[0]
print(f'The number of features with missing values is {missing_values}')

The number of features with missing values is 0


In [13]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical_ = ['number_of_courses_viewed', 'interaction_count', 'lead_score', 'annual_income']

In [14]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(Data, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [15]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [16]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [17]:
y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_test = df_test['converted'].values

In [18]:
len(y_train)

876

In [19]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

## Question 1: ROC AUC feature importance
ROC AUC could also be used to evaluate feature importance of numerical variables.

Let's do that

For each numerical variable, use it as score (aka prediction) and compute the AUC with the y variable as ground truth.
Use the training dataset for that
If your AUC is < 0.5, invert this variable by putting "-" in front

(e.g. -df_train['balance'])

AUC can go below 0.5 if the variable is negatively correlated with the target variable. You can change the direction of the correlation by negating this variable - then negative correlation becomes positive.

Which numerical variable (among the following 4) has the highest AUC?

lead_score
number_of_courses_viewed
interaction_count
annual_income

In [ ]:
from sklearn.metrics import roc_auc_score

aucs = {}

for n in numerical_:
    score = df_train[n] 
    auc = roc_auc_score(y_train,score)
    if auc < 0.5:
        auc = roc_auc_score(y_train, -score)
    aucs[n] = auc

print(aucs)
aucs.

{'number_of_courses_viewed': 0.7635680590007088, 'interaction_count': 0.738270176293409, 'lead_score': 0.6144993577250176, 'annual_income': 0.5519578313253012}


In [ ]:
print(f'Answer of q1 is : {}')